In [ ]:
import os
import sys
import glob
import math
import pandas as pd
import numpy as np
import itertools
import random
import requests
import xml.etree.ElementTree as ET
import csv

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.patches as patches

#import tensorflow_probability as tfp
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import EarlyStopping
from IPython.display import SVG

from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, normalize
from sklearn.preprocessing import MinMaxScaler
import warnings

warnings.filterwarnings('ignore')

from ipywidgets import FloatProgress
from IPython.display import display

fig_width = 12
plt.rcParams["font.size"] = 40
plt.rcParams['axes.labelsize'] = 40
plt.rcParams['axes.labelweight'] = 'bold'
#plt.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
#plt.rcParams.keys()

from codes.utils.load_utils import num_labels, time_change, setup_dir, find_xml_filenames, find_csv_filenames
import extendedMD as emd

In [ ]:
#data = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori\2022\labeled\LB01_l_acc.csv")
data = pd.read_csv(r"G:\JaimeMorales\Codes\omizunagidori\database\omizunagidori\2022\l_2022_acc.csv")

In [ ]:
data

In [ ]:
data["timestamp"] = pd.to_datetime(data["timestamp"],unit='ms')

In [ ]:
dates = []

In [ ]:
for i in range(len(data)):
    date = str(data['timestamp'][i].date())
    dates.append(date)

In [ ]:
data['date'] = dates

In [ ]:
data

In [ ]:
data26 = data[data['date']=='2022-08-26']
data27 = data[data['date']=='2022-08-27']
data28 = data[data['date']=='2022-08-28']

In [ ]:
data26.drop(["timestamp","activity_class","label","date"],axis=1,inplace=True)
data27.drop(["timestamp","activity_class","label","date"],axis=1,inplace=True)
data28.drop(["timestamp","activity_class","label","date"],axis=1,inplace=True)

In [ ]:
new_data = data26.iloc[0:200000]

In [ ]:
motif_candidates_dic_list, ts_1d = emd.find_motifs_from_emd(new_data, 30, 10, 10, 10, adaptive_break_points=True, z_threshold=0.01)

In [ ]:
len(ts_1d)

In [ ]:
len(motif_candidates_dic_list[1]['members_ts_pointers'][0])

In [ ]:
motif_candidates_dic_list[2]